In [39]:
# Filter out non-video related events 
import os
import glob
import json
import csv
import re

curdir=os.getcwd()

jsonfiles=glob.iglob(curdir+'/K-MOOC tracking_log_jsonized'+'/*') #input, change this path
targeteventlist='targetevent.csv' #input

targeteventfile='videodata.json' #output

with open(targeteventlist,'r',encoding='utf-8') as rf:
    reader=csv.reader(rf)
    targetevents=[]
    next(reader)
    for row in reader:
        targetevents.append(row[0])
    rf.close()

#print(targetevents)
    
data=[]
#count=0
eventcount=0
for jsonfile in jsonfiles:
#    count=count+1
#    if (count%50==0):
#        print(count)
    wholeevents=json.load(open(jsonfile,'r',encoding='utf-8'))
    for event in wholeevents:
        eventtype=event["event_type"]
        if(eventtype in targetevents):
            # Note that for these video-related events, the "event" field is ill-formatted.
            # e.g., "event": "{\"code\": \"html5\", \"type\": \"onSlideSeek\", \"id\": \"8dab4b16f5fa467da92b5f01e6eb0ac6\", \"new_time\": 471, \"old_time\": 396.613652}"
            # So we re-format this field by nesting. 
            neweventvalue={}
            oldeventvalue=[x for x in re.split('{|}|, |"|\"|: ',event["event"]) if x]
            npairs=int(len(oldeventvalue)/2)
            for i in range(npairs):
                key=oldeventvalue[2*i]
                value=oldeventvalue[2*i+1]
                numval=value.replace('.','',1)
                if(numval.isdigit()):
                    value=float(value)
                neweventvalue[key]=value                        
            event["event"]=neweventvalue            
            data.append(event)
            eventcount=eventcount+1

            
with open(targeteventfile,'w',encoding='utf-8') as wf:
    eventwriter=csv.writer(wf,delimiter=',')
    json.dump(data,wf, sort_keys=True, indent=4)
    wf.close()

#print(eventcount)

# videodata.json --> 536MB                
# eventcount = 435961

50
100
150
200
250
300


In [40]:
# Split into smaller files (Tableau can read files with size upto 128MB)
linesperfile=int(eventcount/5)
for i in range(5):
    smallfilename='videodata_'+str(i)+'.json'
    with open(smallfilename,'w',encoding='utf-8') as sf:
        lidx=i*linesperfile
        ridx=min((i+1)*linesperfile, eventcount)
        json.dump(data[lidx:ridx], sf, sort_keys=True, indent=4)
        sf.close()
        